In [1]:
import pandas as pd
import geopandas as gpd

In [3]:
od_files_df = pd.DataFrame([
    ['TDM','data/WFv910_BY_2019_DISTSML_Daily_AutoTransit.csv'],
    ['E2.7.3','data/WFv920-E2.7.3_BY_2019_DISTSML_Daily_AutoTransit.csv'],
    ['E2.13.5','data/WFv920-E2.13.5_BY_2019_DISTSML_Daily_AutoTransit.csv'],
    ['E2.14.4','data/WFv920-E2.14.4_BY_2019_DISTSML_Daily_AutoTransit.csv']
], columns=['investigation','filename'])

taz_df = pd.read_csv("geodata/WFv910_TAZ.csv")

In [6]:
# Carregar o arquivo GeoJSON
gdf = gpd.read_file("geodata/TAZ/WFv910_TAZ.shp")

# Criar uma nova coluna concatenando os valores de DISTSML, DISTMED e DISTLRG para dissolução
gdf["DISSOLVE_KEY"] = gdf["DISTSML"].astype(str) + "_" + gdf["DISTMED"].astype(str) + "_" + gdf["DISTLRG"].astype(str)

# Dissolver os polígonos com base na coluna combinada
gdf_dissolved = gdf.dissolve(by="DISSOLVE_KEY")

dist_sml_df = gdf_dissolved[['DISTSML','DISTMED','DISTLRG']].reset_index().drop(columns=['DISSOLVE_KEY'])
dist_sml_df

,DISTSML,DISTMED,DISTLRG
0,100,58,21
1,101,58,21
2,102,59,21
3,103,59,21
4,104,59,21
...,...,...,...
126,96,57,20
127,97,58,21
128,98,58,21
129,99,58,21


In [ ]:
od_data_dcrt_df = pd.DataFrame()

for _, od_file in od_files_df.iterrows():
    print(od_file['filename'])
    _df = pd.read_csv(od_file['filename'])

    # only import Production side of PA pair
    _df = _df[_df['PA']=='P']

    _df['investigation'] = od_file['investigation']
    
    # Select all columns that contain 'dCRT'
    dcrt_columns = [col for col in _df.filter(like='dCRT').columns if 'wdCRT' not in col]
    
    _df.rename(columns={'DISTSML':'p_DISTSML','DISTSML2':'a_DISTSML'}, inplace=True)

    # Include the necessary columns along with the dCRT-related ones
    selected_columns = ['investigation', 'p_DISTSML', 'a_DISTSML'] + dcrt_columns

    _df = _df[selected_columns]

    # Fazer o melt mantendo o índice original e criando uma nova coluna 'purpose'
    _df_melted = _df.melt(id_vars=[col for col in _df.columns if col not in dcrt_columns], 
                          value_vars=dcrt_columns, 
                          var_name='purpose', 
                          value_name='trips')

    # Extrair a parte antes de '_' para a coluna 'purpose'
    _df_melted['purpose'] = _df_melted['purpose'].str.split("_").str[0]

    od_data_dcrt_df = pd.concat([od_data_dcrt_df, _df_melted], ignore_index=True)

    print(_df_melted['trips'].sum()/2)

od_data_dcrt_df

TDM
11249.35
E2.7.3
12321.165000000003
E2.13.5
12379.165
E2.14.4
12301.920000000004


,investigation,p_DISTSML,a_DISTSML,purpose,trips
0,TDM,1,1,HBW,0.0
1,TDM,1,2,HBW,0.0
2,TDM,1,3,HBW,0.0
3,TDM,1,4,HBW,0.0
4,TDM,1,5,HBW,0.0
...,...,...,...,...,...
343215,E2.14.4,131,127,ALL,0.0
343216,E2.14.4,131,128,ALL,0.0
343217,E2.14.4,131,129,ALL,0.0
343218,E2.14.4,131,130,ALL,0.0


In [32]:
_df_melted

,investigation,p_DISTSML,a_DISTSML,purpose,tripos
0,TDM,1,1,HBW,0.0
1,TDM,1,2,HBW,0.0
2,TDM,1,3,HBW,0.0
3,TDM,1,4,HBW,0.0
4,TDM,1,5,HBW,0.0
...,...,...,...,...,...
85800,TDM,131,127,ALL,0.0
85801,TDM,131,128,ALL,0.0
85802,TDM,131,129,ALL,0.0
85803,TDM,131,130,ALL,0.0


In [17]:
# Merge on p_DISTSML
_df = od_data_dcrt_df.merge(
    dist_sml_df[['DISTSML', 'DISTMED', 'DISTLRG']], 
    left_on='p_DISTSML', right_on='DISTSML', 
    how='left', suffixes=('', '_p')
)

# Rename columns for p_DISTSML
_df.rename(columns={'DISTMED': 'p_DISTMED', 'DISTLRG': 'p_DISTLRG'}, inplace=True)

# Merge again on a_DISTSML
_df = _df.merge(
    dist_sml_df[['DISTSML', 'DISTMED', 'DISTLRG']], 
    left_on='a_DISTSML', right_on='DISTSML', 
    how='left', suffixes=('', '_a')
)

# Rename columns for a_DISTSML
_df.rename(columns={'DISTMED': 'a_DISTMED', 'DISTLRG': 'a_DISTLRG'}, inplace=True)

# Drop redundant DISTSML columns
_df.drop(columns=['DISTSML'], inplace=True, errors='ignore')

# Assign to final dataframe
od_data_dcrt_merge_df = _df

# Display the result
display(od_data_dcrt_merge_df)


,investigation,p_DISTSML,a_DISTSML,HBW_dCRT,HBW_wdCRT,HBC_dCRT,HBC_wdCRT,HBO_dCRT,HBO_wdCRT,NHB_dCRT,NHB_wdCRT,ALL_dCRT,ALL_wdCRT,p_DISTMED,p_DISTLRG,DISTSML_a,a_DISTMED,a_DISTLRG
0,TDM,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,TDM,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0
2,TDM,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,3.0,1.0
3,TDM,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,3.0,1.0
4,TDM,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69691,E2.14.4,131,127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,127.0,2.0,1.0
69692,E2.14.4,131,128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,128.0,7.0,3.0
69693,E2.14.4,131,129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,129.0,17.0,7.0
69694,E2.14.4,131,130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,130.0,25.0,11.0


In [18]:
od_data_dcrt_distsml_df = od_data_dcrt_df.groupby(['DISTSML','a_DISTSML'], as_index=False).agg(TRIPS=('linked_weight_adj','sum'))
od_data_dcrt_distsml_df

KeyError: 'DISTSML'

# Prepare OBS Data

In [39]:
# Load OBS Data
obs_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)
obs_df

,id,date_completed,final_route_surveyed_code,final_route_surveyed,direction,mode,purpose,resident_or_visitor_code,home_or_hotel_address,home_or_hotel_city,...,FirstOn_lat,FirstOn_lon,LastOff_lat,LastOff_lon,p_Stop_lat,p_Stop_lon,a_Stop_lat,a_Stop_lon,unlinked_weight_adj,linked_weight_adj
0,22.0,9-Sep-19,SLC_1_704_00,TRAX GREEN LINE TO WEST VALLEY CENTRAL STATION,0.0,1.0,1.0,1.0,5458 Jeremiah Dr,Kearns,...,40.783280,-111.980172,40.667658,-112.020344,40.667658,-112.020344,40.783280,-111.980172,2.344,0.781
1,24.0,9-Sep-19,SLC_1_704_00,TRAX GREEN LINE TO WEST VALLEY CENTRAL STATION,0.0,1.0,4.0,2.0,265 E 8880 S,Sandy,...,40.783280,-111.980172,40.769343,-111.900051,40.783280,-111.980172,40.769343,-111.900051,12.733,12.733
2,29.0,9-Sep-19,SLC_1_704_00,TRAX GREEN LINE TO WEST VALLEY CENTRAL STATION,0.0,1.0,3.0,1.0,644 W North Temple St,Salt Lake City,...,40.771578,-111.904318,40.683817,-111.986704,40.771578,-111.904318,40.683817,-111.986704,7.591,3.795
3,30.0,9-Sep-19,SLC_1_703_00,TRAX RED LINE TO DAYBREAK PARKWAY STATION,0.0,1.0,2.0,1.0,262 E Whitlock Ave,South Salt Lake,...,40.724091,-111.896892,40.280140,-111.725489,40.724091,-111.896892,40.280140,-111.725489,11.440,5.720
4,31.0,9-Sep-19,SLC_1_704_01,TRAX GREEN LINE TO AIRPORT STATION,1.0,1.0,1.0,1.0,2900 Lehman Ave,West Valley City,...,40.694361,-111.959834,40.763855,-111.891071,40.694361,-111.959834,40.763855,-111.891071,6.672,6.672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# only include drive to CRT trips
_df = obs_df[obs_df['v83_MC_txt']=='Drive_CRT']

# only include select columns
_df = _df[['v83_MC_txt','p_TAZID','a_TAZID','linked_weight_adj']]

# join p_TAZID and a_TAZID to TAZID in taz_df and get DISTSML, DISTMED, DISTLRG 

# Sample merge to join on p_TAZID
_df = _df.merge(taz_df[['TAZID', 'DISTSML', 'DISTMED', 'DISTLRG']], 
                  left_on='p_TAZID', right_on='TAZID', how='left', suffixes=('', '_p'))

# Rename columns
_df.rename(columns={'DISTSML': 'p_DISTSML', 'DISTMED': 'p_DISTMED', 'DISTLRG': 'p_DISTLRG'}, inplace=True)

# Merge again to join on a_TAZID
_df = _df.merge(taz_df[['TAZID', 'DISTSML', 'DISTMED', 'DISTLRG']], 
              left_on='a_TAZID', right_on='TAZID', how='left', suffixes=('', '_a'))

# Rename columns
_df.rename(columns={'DISTSML': 'a_DISTSML', 'DISTMED': 'a_DISTMED', 'DISTLRG': 'a_DISTLRG'}, inplace=True)

# Drop redundant TAZID columns
_df.drop(columns=['TAZID', 'TAZID_a'], inplace=True, errors='ignore')

obs_od_data_dcrt_df = _df

display(obs_od_data_dcrt_df)
# check trip numbers should be around 15,000?
obs_od_data_dcrt_df['linked_weight_adj'].sum()

,v83_MC_txt,p_TAZID,a_TAZID,linked_weight_adj,p_DISTSML,p_DISTMED,p_DISTLRG,a_DISTSML,a_DISTMED,a_DISTLRG
0,Drive_CRT,1457.0,2848.0,5.720,56.0,34.0,14.0,101.0,58.0,21.0
1,Drive_CRT,2689.0,1035.0,9.068,92.0,55.0,20.0,46.0,32.0,13.0
2,Drive_CRT,2493.0,1077.0,4.656,88.0,53.0,20.0,46.0,32.0,13.0
3,Drive_CRT,1968.0,2939.0,1.379,68.0,42.0,16.0,103.0,59.0,21.0
4,Drive_CRT,2931.0,1051.0,1.697,103.0,59.0,21.0,54.0,33.0,14.0
...,...,...,...,...,...,...,...,...,...,...
1973,Drive_CRT,374.0,1035.0,11.951,15.0,12.0,5.0,46.0,32.0,13.0
1974,Drive_CRT,283.0,1080.0,5.976,12.0,10.0,4.0,46.0,32.0,13.0
1975,Drive_CRT,827.0,1035.0,8.815,32.0,23.0,9.0,46.0,32.0,13.0
1976,Drive_CRT,708.0,1034.0,5.976,27.0,20.0,8.0,46.0,32.0,13.0


14735.382999999998

In [44]:
obs_od_data_dcrt_distsml_df = obs_od_data_dcrt_df.groupby(['p_DISTSML','a_DISTSML'], as_index=False).agg(TRIPS=('linked_weight_adj','sum'))
obs_od_data_dcrt_distsml_df

,p_DISTSML,a_DISTSML,TRIPS
0,3.0,38.0,3.801
1,3.0,46.0,18.674
2,3.0,54.0,4.224
3,4.0,32.0,2.805
4,4.0,38.0,4.209
...,...,...,...
900,111.0,54.0,7.161
901,111.0,66.0,14.273
902,111.0,78.0,14.273
903,119.0,46.0,5.514


In [46]:
obs_od_data_dcrt_distmed_df = obs_od_data_dcrt_df.groupby(['p_DISTMED','a_DISTMED'], as_index=False).agg(TRIPS=('linked_weight_adj','sum'))
obs_od_data_dcrt_distmed_df

,p_DISTMED,a_DISTMED,TRIPS
0,3.0,23.0,2.805
1,3.0,29.0,8.010
2,3.0,32.0,43.297
3,3.0,33.0,21.442
4,4.0,29.0,6.336
...,...,...,...
469,62.0,33.0,9.029
470,62.0,42.0,14.273
471,62.0,46.0,14.273
472,67.0,32.0,5.514


In [45]:
obs_od_data_dcrt_distlrg_df = obs_od_data_dcrt_df.groupby(['p_DISTLRG','a_DISTLRG'], as_index=False).agg(TRIPS=('linked_weight_adj','sum'))
obs_od_data_dcrt_distlrg_df

,p_DISTLRG,a_DISTLRG,TRIPS
0,1.0,9.0,2.805
1,1.0,12.0,14.346
2,1.0,13.0,49.218
3,1.0,14.0,21.442
4,4.0,1.0,4.021
...,...,...,...
142,22.0,17.0,33.920
143,22.0,20.0,96.063
144,22.0,21.0,27.181
145,25.0,12.0,3.657
